In [1]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 950.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of gcsfs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.6/191.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.7 MB/s eta 0:00:00
   ━

In [1]:
import warnings
warnings.filterwarnings('ignore')
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec


import os
import time
import torch
from tqdm.auto import tqdm
import pandas as pd

## Load the dataset

In [2]:
df = pd.read_csv('shoes-clear.csv')
# df['imageURLs'].fillna('https://salonlfc.com/wp-content/uploads/2018/01/image-not-found-1-scaled-1150x647.png', inplace=True)

# print(df[['id','brand','colors','features','imageURLs']])
shoes = []
selected_columns = df[['id','brand','colors','features','imageURLs']]
for i, row in selected_columns.iterrows():
  text = f"{row['brand']} {row['colors']} {row['features']}"
  shoes.extend(text)

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# query= "Which shoe is best for running ? "
# xq = embedding_model.encode(query)
# xq.shape

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [3]:
pinecone = Pinecone(api_key='d130d359-d4c6-4ce9-b5c8-4e25368899fe')
INDEX_NAME = 'shoe'

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    print(f'Deleting index {INDEX_NAME}...')
    pinecone.delete_index(INDEX_NAME)
print(INDEX_NAME)
print('Creating Index')
pinecone.create_index(name=INDEX_NAME,
    dimension=embedding_model.get_sentence_embedding_dimension(),
    metric='cosine',
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) )

index = pinecone.Index(INDEX_NAME)
batch_size = 10
vector_limit = 300
df = df.iloc[:vector_limit]

for i in tqdm(range(0, len(df), batch_size)):
    # Find end of batch
    i_end = min(i + batch_size, len(df))

    # Create IDs batch (use indices from the DataFrame directly)
    ids = [str(x) for x in df.index[i:i_end]]

    # Get the descriptions for embedding
    names = df['name'].iloc[i:i_end].tolist()

    images = df['imageURLs'].iloc[i:i_end].tolist()
    prices = df['prices.amountMax'].iloc[i:i_end].tolist()
    descriptions = [f"{name} {image} {price}" for name, image, price in zip(names, images, prices)]


    # Create embeddings
    xc = embedding_model.encode(descriptions)
    metadata = [{'name': name,'image':image,'price':price} for name,image,price in zip(names,images,prices)]
    # Create records list for upsert (you can also include brands if needed)
    records = zip(ids, xc,metadata)
    # Upsert to Pinecone
    index.upsert(vectors=records)
print('Done adding shoe embeddings to our Vector DB, you may now run the quer')

Deleting index shoe...
shoe
Creating Index


  0%|          | 0/30 [00:00<?, ?it/s]

Done adding shoe embeddings to our Vector DB, you may now run the quer
